In [10]:
from pathlib import Path
import os
import sys
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))
from utils.crossvalidation import xval
from settings.paths import match_path, img_path, rf_path, bnn_path, flex_path, data_path
import pandas as pd
import matplotlib.pyplot as plt
from utils.preprocessing import create_bins, prep_data, rename_aper
from settings.columns import specz, calculate_colors, create_colors, list_feat
from utils.metrics import nmad, bias ,out_frac, rmse, mse
import warnings
import numpy as np
from utils.metrics import nmad, out_frac, bias, rmse
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from cycler import cycler
from sklearn.model_selection import GridSearchCV
from utils.preprocessing import create_bins
from utils.crossvalidation import metric_per_bin
from utils.preprocessing import create_bins, rename_aper, mag_redshift_selection, prep_wise, missing_input, flag_observation

plt.rcParams["font.size"] = 22
blue = (0, 0.48, 0.70)
orange = (230/255,159/255, 0)
yellow = (0.94, 0.89, 0.26)
pink = (0.8, 0.47, 0.65)
CB_color_cycle = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#e41a1c', '#dede00']
plt.rcParams['axes.prop_cycle'] = cycler('color', CB_color_cycle)

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from utils.correct_extinction import correction
aper = "PStotal"

In [ ]:
table_dr2 = pd.read_table(os.path.join(match_path,"STRIPE82_DR2_DR16Q1a_unWISE2a_GALEXDR672a.csv"), sep=",")
table_dr2 = rename_aper(table_dr2)

In [3]:
table_dr4 = pd.read_table(os.path.join(match_path,"STRIPE82_DR4_DR16Q1a_unWISE2a_GALEXDR672a.csv"), sep=",")
table_dr4 = rename_aper(table_dr4)

In [4]:
len(table_dr4)

37978

In [4]:
feat_mag = list_feat(aper=aper, broad = True, narrow = True, galex = True, wise = True)
feat = create_colors(broad = True, narrow=True, wise=True, galex=True, aper=aper)
# data = table_corrected.copy(deep=True)

In [5]:
data = mag_redshift_selection(table_dr4, rmax=22, zmax=5)

In [6]:
data = prep_wise(data)


In [7]:
data = flag_observation(data)

In [8]:
data = correction(data)

In [9]:
data = missing_input(data)


In [42]:
data = calculate_colors(data, broad = True, narrow= True, wise = True, galex= True, aper=aper)
data, bins, itvs = create_bins(data = data, bin_size=0.5, return_data = True, var = specz)

In [79]:
list_ids = sample["ID_1"].apply(lambda x: x.split(" ")[0].split("-")[-1])

In [72]:
# Did two steps of train_test_split just to ensure that some specific targets (9) are in the test set
# These 9 targets were originally used to plot some PDFs in thr first draft of the paper

# sample = pd.read_csv(os.path.join(data_path, "sample_plot_paper_9.txt"), delim_whitespace=True)
# list_ids = sample["ID_1"].apply(lambda x: x.split(" ")[0].split("-")[-1])
# check=False
# k=0
# while check==False:
#     train, test = train_test_split(data, test_size=0.5, random_state=823, stratify=data['Zclass'])
#     train2, test2 = train_test_split(test, test_size=0.5, random_state=124, stratify=test['Zclass'])
#     list_ids_test = test2["ID"].apply(lambda x: x.split(" ")[0].split("-")[-1])
#     compare = [True if i in list_ids_test.values else False for i in list_ids.values]
#     print(compare)
#     check = all(compare)
#     k+=1

[True, True, True, True, True, True, True, True, True]


In [78]:
split1, split2 = train_test_split(data, test_size=0.5, random_state=823, stratify=data['Zclass'])
split3, test = train_test_split(split2, test_size=0.5, random_state=124, stratify=split2['Zclass'])

train = pd.concat([split1, split2, split3])

In [81]:
# list_ids_test = test["ID"].apply(lambda x: x.split(" ")[0].split("-")[-1])
# compare = [True if i in list_ids_test.values else False for i in list_ids.values]

In [11]:
# train, test = train_test_split(data, random_state=22, stratify=data['Zclass'])
zclass_train = train["Zclass"]

In [ ]:
# Trying to find the random_state where the testing set contains the 9 objects from sample_plot_paper.txt


In [12]:
train.Z.describe().T

count    24863.000000
mean         1.650779
std          0.743512
min          0.034357
25%          1.098107
50%          1.574127
75%          2.120000
max          4.920000
Name: Z, dtype: float64

In [13]:
test.Z.describe().T

count    8288.000000
mean        1.653930
std         0.740635
min         0.060000
25%         1.097735
50%         1.574966
75%         2.119848
max         4.859451
Name: Z, dtype: float64

In [14]:
data.flag_WISE.value_counts()

0    31146
1     2005
Name: flag_WISE, dtype: int64

In [15]:
def fraction_of_non_observation(df, survey="WISE"):
    if survey=="WISE":
        print(len(df[df["objID_x"].isna()])/len(df))
        return 
    
    if survey=="GALEX":
        print(len(df[df["name"].isna()])/len(df))
        return 
    
def fraction_of_non_detection(df, survey="WISE"):
    if survey=="WISE":
        print("W1:", len(df[df["W1"].isna()])/len(df))
        print("W2:", len(df[df["W2"].isna()])/len(df))
        return 
    
    if survey=="GALEX":
        print("FUV:", len(df[df["FUVmag"].isna()].dropna(subset=["name"]))/len(df.dropna(subset=["name"])))
        print("NUV:", len(df[df["NUVmag"].isna()].dropna(subset=["name"]))/len(df.dropna(subset=["name"])))
        return

In [73]:
#number of objects with non-observation in GALEX
len(data[data["name"].isna()])

#number of objects with non-observation in WISE
len(data[data["objID_x"].isna()])

2005

In [192]:
fraction_of_non_observation(data, survey="WISE")
fraction_of_non_observation(train, survey="WISE")
fraction_of_non_observation(test, survey="WISE")

0.060480830140870565
0.060129509713228495
0.06153474903474904


In [193]:
fraction_of_non_observation(data, survey="GALEX")
fraction_of_non_observation(train, survey="GALEX")
fraction_of_non_observation(test, survey="GALEX")

0.6919851588187385
0.6918312351687246
0.6924469111969112


In [194]:
fraction_of_non_detection(data, survey="WISE")
fraction_of_non_detection(train, survey="WISE")
fraction_of_non_detection(test, survey="WISE")

W1: 0.06922868088443787
W2: 0.14132303701245816
W1: 0.06901821984474922
W2: 0.14097252946144873
W1: 0.06986003861003862
W2: 0.14237451737451737


In [195]:
fraction_of_non_detection(data, survey="GALEX")
fraction_of_non_detection(train, survey="GALEX")
fraction_of_non_detection(test, survey="GALEX")

FUV: 0.6410733522671629
NUV: 0.0
FUV: 0.6378230227094753
NUV: 0.0
FUV: 0.6508434680266771
NUV: 0.0


In [16]:
flags = ["flag_WISE", "flag_GALEX"]

In [17]:
feat = {}
feat["broad"] =  create_colors(broad = True, narrow=False, wise=False, galex=False, aper=aper)
feat["broad"] = feat["broad"]+flags
feat["broad+narrow"]=  create_colors(broad = True, narrow=True, wise=False, galex=False, aper=aper)
feat["broad+narrow"]=   feat["broad+narrow"]+flags

feat["broad+GALEX+WISE"]=  create_colors(broad = True, narrow=False, wise=True, galex=True, aper=aper)
feat["broad+GALEX+WISE"]=  feat["broad+GALEX+WISE"]+flags


feat["broad+WISE+narrow"]=  create_colors(broad = True, narrow=True, wise=False, galex=True, aper=aper)
feat["broad+WISE+narrow"]=  feat["broad+WISE+narrow"]+["flag_WISE"]


feat["broad+GALEX+WISE+narrow"]=  create_colors(broad = True, narrow=True, wise=True, galex=True, aper=aper)
feat["broad+GALEX+WISE+narrow"]=  feat["broad+GALEX+WISE+narrow"]+flags

In [18]:
color_feat = {}
color_feat["broad"] = "#377eb8"
color_feat["broad+narrow"] =  "#ff7f00"
color_feat["broad+GALEX+WISE"]= "#4daf4a"
color_feat["broad+GALEX+WISE+narrow"]= "#f781bf"


In [19]:
z={}
z["broad+WISE+narrow"] = xval(train, zclass_train , feat["broad+WISE+narrow"],"broad+WISE+narrow", aper=aper, save_data=True, save_model=False, save_result=False)

KeyboardInterrupt: 

In [ ]:
from settings.paths import validation_path
test.index.name = 'index'
test.to_csv(os.path.join(validation_path, "test_new.csv"))

In [61]:

for key, value in feat.items():
    print(key)
    z[key] = xval(train, zclass_train , feat[key], key, aper=aper, save_model=False, save_result=False)


broad
broad+narrow
broad+GALEX+WISE
broad+WISE+narrow
broad+GALEX+WISE+narrow


In [62]:
for key, value in feat.items():
    print(key)
    rmse_list = []
    sigma_list = []
    bias_list = []
    n30_list = []
    n15_list = []
    for fold in z[key].fold.unique():
        rmse_list.append(rmse(z[key][z[key]["fold"]==fold].z_pred, z[key][z[key]["fold"]==fold].Z))
        sigma_list.append(nmad(z[key][z[key]["fold"]==fold].z_pred, z[key][z[key]["fold"]==fold].Z))
        bias_list.append(bias(z[key][z[key]["fold"]==fold].z_pred, z[key][z[key]["fold"]==fold].Z))
        n30_list.append(out_frac(z[key][z[key]["fold"]==fold].z_pred, z[key][z[key]["fold"]==fold].Z, 0.3))
        n15_list.append(out_frac(z[key][z[key]["fold"]==fold].z_pred, z[key][z[key]["fold"]==fold].Z, 0.15))
    print('RMSE', np.round(np.mean(rmse_list),4), np.round(np.std(rmse_list),4))
    print('NMAD', np.round(np.mean(sigma_list),4),  np.round(np.std(sigma_list),4))
    print('bias', np.round(np.mean(bias_list),4),  np.round(np.std(bias_list),4))
    print('n15', np.round(np.mean(n15_list),4),  np.round(np.std(n15_list),4))
    print('n30', np.round(np.mean(n30_list),4),  np.round(np.std(n30_list),4))
    

broad
RMSE 0.5877 0.0029
NMAD 0.1835 0.0015
bias -0.0012 0.0052
n15 0.4274 0.0067
n30 0.1643 0.0016
broad+narrow
RMSE 0.5371 0.0026
NMAD 0.1543 0.0027
bias -0.0014 0.006
n15 0.3731 0.0042
n30 0.1372 0.002
broad+GALEX+WISE
RMSE 0.4248 0.0056
NMAD 0.1016 0.0014
bias 0.0016 0.0036
n15 0.2264 0.0049
n30 0.0627 0.0021
broad+WISE+narrow
RMSE 0.5255 0.0037
NMAD 0.1424 0.0033
bias -0.0029 0.0066
n15 0.3548 0.0034
n30 0.1271 0.0037
broad+GALEX+WISE+narrow
RMSE 0.4096 0.0067
NMAD 0.0917 0.0013
bias -0.0007 0.0036
n15 0.2163 0.0057
n30 0.0615 0.0023
